### Tasks:
- clean data (again):
    - impute missing BMI
    - recode 'Lowest_Recent_Hypo_Cat'  
    - recode 'Median_Recent_Hypo_Cat'
-  One-hot encoding for categorical variables
-  apply Kfold CV
- check confusion matrix for female and male

Demographic Gender in general (before build model):

| osteo_predict | 1.0	 | 0.0 |
| --- | --- | --- |
| Female | 24,547 | 24,551 |
| Male | 3,400 | 3,400 |

Confusion matrix in general:

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 1,854 | 962 |
| reality_False | 706 | 2,067|

Confusion matrix for __FEMALE__

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 1,617 | 840 |
| reality_False | 612 | 1,838|

Recall for female: 0.7

Confusion matrix for __MALE__

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 237 | 122 |
| reality_False | 94 | 229|

Recall for male: 0.68

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
# import csv
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [2]:
import os
path = '/Users/thanhng/osteo_experiment/'
os.chdir(path) 

data_osteo = pd.read_csv('osteo_nodup.csv') 
data_fract = pd.read_csv('fract_nodup.csv')

print(data_osteo.shape)
print(data_fract.shape)  

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(55898, 188)
(88779, 190)


In [167]:
#EDA in entire dataset
gender = pd.crosstab(index=data_osteo["osteo_predict"], columns=data_osteo["sex"], margins = True)
gender

sex,F,M,All
osteo_predict,,,
0.0,24551,3400,27951
1.0,24547,3400,27947
All,49098,6800,55898


### Impute missing in BMI

In [36]:
#check missing in BMI
data_osteo['BMI_impute'].isna().sum()

38430

In [3]:
mean_BMI = data_osteo['BMI_Avg'].mean() 
mean_BMI

#BMI impute
data_osteo['BMI_impute'] = data_osteo['BMI_Avg'].copy()
count_extreme = 0
# BMI_impute = []
for i in range(0, len(data_osteo['BMI_impute'])):
    if data_osteo['BMI_impute'][i] > 100 :
        count_extreme +=1
        data_osteo['BMI_impute'][i] = mean_BMI

    if data_osteo['BMI_impute'][i] < 11:
        count_extreme +=1
        data_osteo['BMI_impute'][i] = mean_BMI

data_osteo['BMI_impute'].fillna(0, inplace=True) 
data_osteo['BMI_impute'].describe()

# print(count_extreme)

/Users/thanhng/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/thanhng/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


count    55898.000000
mean         8.756985
std         13.621088
min          0.000000
25%          0.000000
50%          0.000000
75%         22.273403
max         96.150000
Name: BMI_impute, dtype: float64

In [71]:
data_osteo['BMI_impute'].isna().sum()

0

In [130]:
data_osteo.shape

(55898, 189)

### recode Lowest_Recent_Hypo_Cat

In [89]:
#old value before recode
data_osteo['Lowest_Recent_Hypo_Cat'].value_counts()

2.0    2649
3.0     634
4.0     211
1.0       6
Name: Lowest_Recent_Hypo_Cat, dtype: int64

In [4]:
#recode
data_osteo['Lowest_Recent_Hypo_Cat_edit'] = data_osteo['Lowest_Recent_Hypo_Cat'].copy()

data_osteo['Lowest_Recent_Hypo_Cat_edit'] = data_osteo['Lowest_Recent_Hypo_Cat_edit'].replace(1, 0)
data_osteo['Lowest_Recent_Hypo_Cat_edit'] = data_osteo['Lowest_Recent_Hypo_Cat_edit'].replace(2, 1)
data_osteo['Lowest_Recent_Hypo_Cat_edit'] = data_osteo['Lowest_Recent_Hypo_Cat_edit'].replace(3, 2)
data_osteo['Lowest_Recent_Hypo_Cat_edit'] = data_osteo['Lowest_Recent_Hypo_Cat_edit'].replace(4, 3)    

data_osteo['Lowest_Recent_Hypo_Cat_edit'].fillna(0, inplace=True) 
data_osteo['Lowest_Recent_Hypo_Cat_edit'].value_counts()


0.0    52404
1.0     2649
2.0      634
3.0      211
Name: Lowest_Recent_Hypo_Cat_edit, dtype: int64

### recode 'Median_Recent_Hypo_Cat'

In [91]:
print(data_osteo['Median_Recent_Hypo_Cat'].value_counts())
print('NA: ', data_osteo['Median_Recent_Hypo_Cat'].isna().sum())

2.0    1610
1.0    1496
3.0     327
4.0      67
Name: Median_Recent_Hypo_Cat, dtype: int64
NA:  52398


In [5]:
#copy original
data_osteo['Median_Recent_Hypo_Cat_edit'] = data_osteo['Median_Recent_Hypo_Cat'].copy()
#recode
data_osteo['Median_Recent_Hypo_Cat_edit'] = data_osteo['Median_Recent_Hypo_Cat_edit'].replace(4,3)
data_osteo['Median_Recent_Hypo_Cat_edit'] = data_osteo['Median_Recent_Hypo_Cat_edit'].replace(3,2)
data_osteo['Median_Recent_Hypo_Cat_edit'] = data_osteo['Median_Recent_Hypo_Cat_edit'].replace(2,1)
data_osteo['Median_Recent_Hypo_Cat_edit'] = data_osteo['Median_Recent_Hypo_Cat_edit'].replace(1,0)

#fill NA 
data_osteo['Median_Recent_Hypo_Cat_edit'].fillna(0, inplace=True) 

In [6]:
# extract needed columns
data_extract = data_osteo.filter(['sex', 'Race_comb', 'Age_combine','osteo_predict', 'Hyponatremia_Prior',
    'Drug_antipsych_prior', 'Drug_Estrogens_prior' 'Drug_Glucocorticoids_prior', 
     'Drug_Nsaids_prior', 'Drug_Opiates_prior', 'Drug_Pp_inhibitors_prior' 'Drug_Progesterone_prior', 
    'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior', 'Drug_Thiazide_prior', 
     'Chronic_Hyponatremia', 'Recent_Hyponatremia', 'Liver_Prior',
     'HeartDisease_Prior', 'PulmDisease_Prior', 'CNS_Disease_Prior', 
     'Malignancy_Prior', 'Alcohol_Prior', 'Tobacco_Prior',
     'Drug_Loop_Diuretic_Prior', 'Sodium_Avg_Ever', 'BMI_impute', 'osteo_predict'
     'Lowest_Recent_Hypo_Cat_edit', 'Median_Recent_Hypo_Cat_edit']) 

#fill NA in sodium by 0
data_extract['Sodium_Avg_Ever'] = data_extract['Sodium_Avg_Ever'].replace(np.NaN, 0) 

In [135]:
data_extract.columns

Index(['sex', 'Race_comb', 'Age_combine', 'osteo_predict',
       'Hyponatremia_Prior', 'Drug_antipsych_prior', 'Drug_Nsaids_prior',
       'Drug_Opiates_prior', 'Drug_Seizure_prior', 'Drug_Ssris_prior',
       'Drug_Tc_antidepress_prior', 'Drug_Thiazide_prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia', 'Liver_Prior',
       'HeartDisease_Prior', 'PulmDisease_Prior', 'CNS_Disease_Prior',
       'Malignancy_Prior', 'Alcohol_Prior', 'Tobacco_Prior',
       'Drug_Loop_Diuretic_Prior', 'Sodium_Avg_Ever', 'BMI_impute',
       'Median_Recent_Hypo_Cat_edit'],
      dtype='object')

In [136]:
data_extract.shape

(55898, 25)

### One-hot encoding

In [137]:
# Copy the original data
data_OHE = data_extract.copy()

numeric_subset = data_extract.select_dtypes('number')
categorical_subset = data_extract.select_dtypes('object')

# One hot encode
categorical_subset = pd.get_dummies(categorical_subset) 

# Join the two dataframes using concat. Make sure to use axis = 1 to perform a column bind
data_OHE = pd.concat([numeric_subset, categorical_subset], axis = 1)

#y variable for OHE
y_OHE = data_OHE['osteo_predict']

In [150]:
data_OHE.columns

Index(['osteo_predict', 'Hyponatremia_Prior', 'Drug_antipsych_prior',
       'Drug_Nsaids_prior', 'Drug_Opiates_prior', 'Drug_Seizure_prior',
       'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior', 'Drug_Thiazide_prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia', 'Liver_Prior',
       'HeartDisease_Prior', 'PulmDisease_Prior', 'CNS_Disease_Prior',
       'Malignancy_Prior', 'Alcohol_Prior', 'Tobacco_Prior',
       'Drug_Loop_Diuretic_Prior', 'Sodium_Avg_Ever', 'BMI_impute',
       'Median_Recent_Hypo_Cat_edit', 'sex_F', 'sex_M',
       'Race_comb_African American', 'Race_comb_Caucasian', 'Race_comb_Other',
       'Age_combine_from_30_to_50', 'Age_combine_from_50_to_70',
       'Age_combine_from_70_to_80', 'Age_combine_less_than_30',
       'Age_combine_more_than_80'],
      dtype='object')

In [151]:
data_OHE.shape

(55898, 32)

### apply CV

In [138]:
y = data_OHE['osteo_predict']
X = data_OHE.drop('osteo_predict', axis=1) 

# prepare cross validation
kf = KFold(n_splits=10)

# enumerate splits
for train_index, test_index in kf.split(X,y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [139]:
print('X.shape: ', X.shape)
print('X_train.shape: ', X_train.shape )
print('X_test.shape: ', X_test.shape) 

X.shape:  (55898, 31)
X_train.shape:  (50309, 31)
X_test.shape:  (5589, 31)


In [140]:
print('y.shape: ', y.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

y.shape:  (55898,)
y_train.shape:  (50309,)
y_test.shape:  (5589,)


### Applying logistics regression

In [141]:
#TRAIN MODEL

import sklearn.linear_model as linear_model
from sklearn.metrics import accuracy_score

cls = linear_model.LogisticRegression(penalty = 'l2', C = i,random_state = 0)
    
#TRAIN model on TRAINING SET
cls.fit(X_train, y_train)

#PREDICT model on TEST SET
y_pred = cls.predict(X_test) 

print('Accuracy of LR on **TESTING SET**: {:.2f}'.format(accuracy_score(y_test, y_pred))) 

Accuracy of LR on **TESTING SET**: 0.70


In [142]:
#confusion matrix in general
confusion_matrix(y_test, y_pred)

array([[1854,  962],
       [ 706, 2067]])

In [143]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.72      0.66      0.69      2816
        1.0       0.68      0.75      0.71      2773

avg / total       0.70      0.70      0.70      5589



### Check errors across gender

In [144]:
gender_err = pd.crosstab(index=X_test["sex_F"], columns=y_pred)
gender_err

col_0,0.0,1.0
sex_F,,
0,331,351
1,2229,2678


In [145]:
#combine X_test, y_test, y_pred into separate df to cal confusion matrix based on gender

testing_data = pd.concat([X_test, y_test], axis=1)
print('before combine y_pred: ', testing_data.shape)

testing_data['y_pred'] = y_pred
print('after: ', testing_data.shape)

before combine y_pred:  (5589, 32)
after:  (5589, 33)


In [146]:
#how many y
testing_data['osteo_predict'].value_counts()

0.0    2816
1.0    2773
Name: osteo_predict, dtype: int64

In [147]:
#how many y_hat
testing_data['y_pred'].value_counts()

1.0    3029
0.0    2560
Name: y_pred, dtype: int64

In [152]:
#how many female in entire dataset
data_OHE['sex_F'].value_counts()

1    49098
0     6800
Name: sex_F, dtype: int64

In [153]:
#how many female/male in testing data
testing_data['sex_M'].value_counts()

0    4907
1     682
Name: sex_M, dtype: int64

In [52]:
#extract only female from testing data
female = testing_data.loc[testing_data['sex_F'] == 1]
print('check missing: ', female['sex_F'].isna().sum())
female['sex_F'].value_counts()


check missing:  0


1    4907
Name: sex_F, dtype: int64

In [154]:
#confusion matrix for female
confusion_matrix(female['osteo_predict'], female['y_pred'])

array([[1617,  840],
       [ 612, 1838]])

In [155]:
from sklearn.metrics import classification_report
print(classification_report(female['osteo_predict'], female['y_pred']))

             precision    recall  f1-score   support

        0.0       0.73      0.66      0.69      2457
        1.0       0.69      0.75      0.72      2450

avg / total       0.71      0.70      0.70      4907



In [156]:
#extract male
male = testing_data.loc[testing_data['sex_M'] == 1]
print('check missing: ', male['sex_M'].isna().sum())
male['sex_M'].value_counts()

check missing:  0


1    682
Name: sex_M, dtype: int64

In [157]:
#confusion matrix for male
confusion_matrix(male['osteo_predict'], male['y_pred'])

array([[237, 122],
       [ 94, 229]])

In [158]:
from sklearn.metrics import classification_report
print(classification_report(male['osteo_predict'], male['y_pred']))

             precision    recall  f1-score   support

        0.0       0.72      0.66      0.69       359
        1.0       0.65      0.71      0.68       323

avg / total       0.69      0.68      0.68       682



## regulation in LR

In [174]:
#TRAIN MODEL

import sklearn.linear_model as linear_model
from sklearn.metrics import accuracy_score

C_param_range = [0.001,0.01,0.1,1,10,100]

cls = linear_model.LogisticRegression(penalty = 'l1', C = i,random_state = 0)
    
#TRAIN model on TRAINING SET
cls.fit(X_train, y_train)

#PREDICT model on TEST SET
y_pred = cls.predict(X_test) 

  
# Saving accuracy score in table
sepal_acc_table = pd.DataFrame(columns = ['C_parameter','Accuracy'])
sepal_acc_table['C_parameter'] = C_param_range
sepal_acc_table.iloc[j,1] = accuracy_score(y_test,y_pred_sepal)
j += 1


print('Accuracy of LR on **TESTING SET**: {:.2f}'.format(accuracy_score(y_test, y_pred))) 

Accuracy of LR on **TESTING SET**: 0.66


## Apply Neural Network

In [71]:
data_OHE.shape

(55898, 32)

In [72]:
# Implement MLP in Keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=31, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, epochs=150, batch_size=20)
# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
50309/50309 [==============================] - 4s 79us/step - loss: 1.2118 - acc: 0.6400
Epoch 2/150
50309/50309 [==============================] - 3s 64us/step - loss: 0.5658 - acc: 0.6961
Epoch 3/150
50309/50309 [==============================] - 3s 64us/step - loss: 0.5584 - acc: 0.7017
Epoch 4/150
50309/50309 [==============================] - 4s 71us/step - loss: 0.5547 - acc: 0.7059
Epoch 5/150
50309/50309 [==============================] - 4s 72us/step - loss: 0.5536 - acc: 0.7055
Epoch 6/150
50309/50309 [==============================] - 3s 65us/step - loss: 0.5538 - acc: 0.7053
Epoch 7/150
50309/50309 [==============================] - 3s 66us/step - loss: 0.5520 - acc: 0.7075
Epoch 8/150
50309/50309 [==============================] - 4s 86us/step - loss: 0.5527 - acc: 0.7060
Epoch 9/150
50309/50309 [==============================] - 4s 72us/step - loss: 0.5522 - acc: 0.7058
Epoch 10/150
50309/50309 [==============================] - 6s 116us/step - loss: 0.5512 - 

In [73]:
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 70.41%


In [91]:
y_pred_NN = model.predict(X_test)
len(y_pred_NN)

5589

In [92]:
y_pred_NN = (y_pred_NN > 0.5)

In [93]:
#combine X_test, y_test, y_pred into separate df to cal confusion matrix based on gender

testing_data_NN = pd.concat([X_test, y_test], axis=1)
print('before combine y_pred: ', testing_data_NN.shape)

testing_data_NN['y_pred'] = y_pred_NN
print('after: ', testing_data_NN.shape)

before combine y_pred:  (5589, 32)
after:  (5589, 33)


In [94]:
#extract only female from testing data
female_NN = testing_data_NN.loc[testing_data_NN['sex_F'] == 1]
print('check missing: ', female_NN['sex_F'].isna().sum())
female_NN['sex_F'].value_counts()

check missing:  0


1    4907
Name: sex_F, dtype: int64

In [161]:
female_NN.columns

Index(['Hyponatremia_Prior', 'Drug_antipsych_prior', 'Drug_Nsaids_prior',
       'Drug_Opiates_prior', 'Drug_Seizure_prior', 'Drug_Ssris_prior',
       'Drug_Tc_antidepress_prior', 'Drug_Thiazide_prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia', 'Liver_Prior',
       'HeartDisease_Prior', 'PulmDisease_Prior', 'CNS_Disease_Prior',
       'Malignancy_Prior', 'Alcohol_Prior', 'Tobacco_Prior',
       'Drug_Loop_Diuretic_Prior', 'Sodium_Avg_Ever', 'BMI_impute',
       'Median_Recent_Hypo_Cat_edit', 'sex_F', 'sex_M',
       'Race_comb_African American', 'Race_comb_Caucasian', 'Race_comb_Other',
       'Age_combine_from_30_to_50', 'Age_combine_from_50_to_70',
       'Age_combine_from_70_to_80', 'Age_combine_less_than_30',
       'Age_combine_more_than_80', 'osteo_predict', 'y_pred'],
      dtype='object')

In [164]:
#confusion matrix of female
confusion_matrix(female['osteo_predict'], female['y_pred']) 

array([[1617,  840],
       [ 612, 1838]])

In [83]:
y_pred_NN.dtype

dtype('float32')

In [165]:
#extract only male from testing data
male_NN = testing_data_NN.loc[testing_data_NN['sex_M'] == 1]
print('check missing: ', male_NN['sex_M'].isna().sum())
male_NN['sex_M'].value_counts()

check missing:  0


1    682
Name: sex_M, dtype: int64

In [166]:
#confusion matrix of female
confusion_matrix(male['osteo_predict'], male['y_pred']) 

array([[237, 122],
       [ 94, 229]])